In [2]:
import pandas as pd
from ekonlpy.tag import Mecab
from multiprocessing import Pool, cpu_count

In [3]:
mecab = Mecab()

In [4]:
def tok_pos(text):
    return mecab.pos(text)

In [8]:
df_corpus = pd.read_csv('../data_combine/corpus_data.csv')
df_mpb = df_corpus[df_corpus['pk'].astype(str).str.startswith('5')]
df_mpb.head()

,pk,Date,Title,Content,Label
115990,520140814001,2014-08-14,금융통화위원회 의사록2014년도 제15차2014814,국내외 경제동향 과 관련하여 일부 위원은 최근 경제의 흐름이 지난 7월 경제전망과 ...,0
115991,520140912001,2014-09-12,금융통화위원회 의사록2014년도 제17차2014912,국내외 경제동향 과 관련하여 일부 위원은 실물경기는 세부부문별로 개선과악화가 계속 ...,0
115992,520140925001,2014-09-25,금융통화위원회 의사록2014년도 제18차2014925,제안설명에 이어 관련부서는 금번 통화신용정책 보고서의 작성과 위원간의견조정 등이 주...,0
115993,520141015001,2014-10-15,금융통화위원회 의사록2014년도 제19차20141015,국내외 경제동향 과 관련하여 일부 위원은 지난 국정감사시 등 당행의 경제전망에 대한...,0
115994,520141023001,2014-10-23,금융통화위원회 의사록2014년도 제20차20141023,제안설명에 이어 관련부서는 보고서 작성의 기본방향 금융안정 현안에대한 주요 분석 내...,1


In [9]:
df_mpb['tokens'] = df_mpb['Content'].apply(tok_pos)

/var/folders/gd/0dt483qd2cg3cqgcsrx3pnkr0000gp/T/ipykernel_20284/1528243245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mpb['tokens'] = df_mpb['Content'].apply(tok_pos)


In [10]:
df_mpb['tokens'].head()

115990    [(국내외, NNG), (경제동향, NNG), (과, JC), (관련, NNG), ...
115991    [(국내외, NNG), (경제동향, NNG), (과, JC), (관련, NNG), ...
115992    [(제안, NNG), (설명, NNG), (에, JKB), (이, VV), (어, ...
115993    [(국내외, NNG), (경제동향, NNG), (과, JC), (관련, NNG), ...
115994    [(제안, NNG), (설명, NNG), (에, JKB), (이, VV), (어, ...
Name: tokens, dtype: object

In [11]:
df_mpb.to_csv('mpb_tokens.csv', index=False)

In [47]:
# 토크나이저 & POS tagging
import pandas as pd
from ekonlpy.tag import Mecab
from multiprocessing import Pool

# Initialize Mecab
mecab = Mecab()

def tokenize_and_pos(text):
    """ Tokenize and perform POS tagging using Mecab. """
    return mecab.pos(text)

def process_in_batches(df, batch_size=1000, output_prefix='processed_batch'):
    """ Process data in batches to handle large datasets and save intermediate results. """
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))
        batch = df.iloc[start:end].copy()  # Create a copy of the slice
        batch['tokens_pos'] = batch['Content'].apply(tokenize_and_pos)
        # Save intermediate results
        batch.to_csv(f'{output_prefix}_{start}.csv', index=False)

def parallel_process(df):
    """ Use parallel processing to speed up the tokenization and POS tagging. """
    with Pool() as pool:
        df['tokens_pos'] = pool.map(tokenize_and_pos, df['Content'])
    return df

def main():
    # Load your dataset (adjust the path to your file)
    df = pd.read_csv("../data_combine/corpus_data.csv")  # Ensure your CSV file has a column named 'Content'

    # Process in batches
    process_in_batches(df, batch_size=1000, output_prefix='processed_batch')

    # Combine all intermediate results into a final file
    import glob
    import os

    # List all batch files
    batch_files = glob.glob('processed_batch_*.csv')

    # Initialize an empty DataFrame for the final result
    final_df = pd.DataFrame()

    # Append each batch file to the final DataFrame
    for batch_file in sorted(batch_files):
        batch_df = pd.read_csv(batch_file)
        final_df = pd.concat([final_df, batch_df], ignore_index=True)

    # Save the final result to a different name
    final_df.to_csv('tokenized.csv', index=False)

    # Optionally, clean up intermediate batch files
    for batch_file in batch_files:
        os.remove(batch_file)

if __name__ == "__main__":
    main()